In [1]:
import numpy as np
import cv2
import os
import PIL
from PIL import ImageTk
import PIL.Image
import speech_recognition as sr
import pyttsx3
from itertools import count
import string
from tkinter import *
import time
try:
    import Tkinter as tk
except:
    import tkinter as tk
import numpy as np
image_x, image_y = 64,64
from keras.models import load_model

#classifier = load_model('asl_detection.h5')

def give_char():
    import numpy as np
    from keras.preprocessing import image
    test_image = image.load_img('tmp1.png', target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    print(result)
    chars="ABCDEFGHIJKMNOPQRSTUVWXYZ"
    indx=  np.argmax(result[0])
    print(indx)
    return(chars[indx])

def check_sim(i,file_map):
    for item in file_map:
        for word in file_map[item]:
            if(i==word):
                return 1,item
    return -1,""

op_dest = r"C:\Users\Pilli Akshitha\OneDrive\Desktop\AD Sign language\AD Sign language\filtered_data\\"
alpha_dest = r"C:\Users\Pilli Akshitha\OneDrive\Desktop\AD Sign language\AD Sign language\alphabet\\"
dirListing = os.listdir(op_dest)
editFiles = []
for item in dirListing:
    if ".webp" in item:
        editFiles.append(item)

file_map={}
for i in editFiles:
    tmp=i.replace(".webp","")
    tmp=tmp.split()
    file_map[i]=tmp

def func(a):
    all_frames=[]
    final= PIL.Image.new('RGB', (380, 260))
    words=a.split()
    for i in words:
        flag,sim=check_sim(i,file_map)
        if(flag==-1):
            for j in i:
                print(j)
                speak_output(j)
                im = PIL.Image.open(alpha_dest+str(j).lower()+"_small.gif")
                frameCnt = im.n_frames
                for frame_cnt in range(frameCnt):
                    im.seek(frame_cnt)
                    im.save("tmp.png")
                    img = cv2.imread("tmp.png")
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (380,260))
                    im_arr = PIL.Image.fromarray(img)
                    for itr in range(15):
                        all_frames.append(im_arr)
        else:
            print(sim)
            speak_output(sim)
            im = PIL.Image.open(op_dest+sim)
            im.info.pop('background', None)
            im.save('tmp.gif', 'gif', save_all=True)
            im = PIL.Image.open("tmp.gif")
            frameCnt = im.n_frames
            for frame_cnt in range(frameCnt):
                im.seek(frame_cnt)
                im.save("tmp.png")
                img = cv2.imread("tmp.png")
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (380,260))
                im_arr = PIL.Image.fromarray(img)
                all_frames.append(im_arr)
                all_frames.append(im_arr)  # Add an additional frame to add pause after each symbol
    final.save("out.gif", save_all=True, append_images=all_frames, duration=100, loop=0)
    return all_frames 

def speak_output(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

img_counter = 0
img_text=''

class Tk_Manage(tk.Tk):
    def __init__(self, *args, **kwargs):     
        tk.Tk.__init__(self, *args, **kwargs)
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand = True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        self.frames = {}
        for F in (StartPage, VtoS):
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame(StartPage)

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

class StartPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Sign Language Translator", font=("Verdana", 12))
        label.pack(pady=10,padx=10)
        button_text_to_sign = tk.Button(self, text="Text to Sign", command=lambda: controller.show_frame(VtoS))
        button_text_to_sign.pack()
        
        # New button for camera detection
        button_camera_detection = tk.Button(self, text="Camera Detection", command=self.start_camera_detection)
        button_camera_detection.pack()

    def start_camera_detection(self):
        # Execute the code from test.py for camera detection
        exec(open("test.py").read())

# VtoS class
class VtoS(tk.Frame):
    def __init__(self, parent, controller):
        cnt=0
        gif_frames=[]
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Text to Sign", font=("Verdana", 12))
        label.pack(pady=10,padx=10)
        gif_box = tk.Label(self)
        
        button1 = tk.Button(self, text="Back to Home",command=lambda: controller.show_frame(StartPage))
        button1.pack()
        
        def gif_stream():
            global cnt
            global gif_frames
            if(cnt==len(gif_frames)):
                return
            img = gif_frames[cnt]
            cnt+=1
            imgtk = ImageTk.PhotoImage(image=img)
            gif_box.imgtk = imgtk
            gif_box.configure(image=imgtk)
            gif_box.after(50, gif_stream)
        
        def hear_voice(inputtxt):
            store = sr.Recognizer()
            with sr.Microphone() as s:
                audio_input = store.record(s, duration=10)
                try:
                    text_output = store.recognize_google(audio_input)
                    inputtxt.insert(END, text_output)
                    # Automatically convert to sign language after voice detection
                    Take_input(inputtxt)
                except:
                    print("Error Hearing Voice")
                    inputtxt.insert(END, '')
                    
        def Take_input(inputtxt):
            INPUT = inputtxt.get("1.0", "end-1c")
            print(INPUT)
            global gif_frames
            gif_frames=func(INPUT)
            global cnt
            cnt=0
            gif_stream()
            gif_box.place(x=400,y=160)
        
        l = tk.Label(self,text = "Enter Text or Voice:")
        l1 = tk.Label(self,text = "OR")
        inputtxt = tk.Text(self, height = 4,width = 25)
        voice_button= tk.Button(self,height = 2,width = 20, text="Record Voice",command=lambda: hear_voice(inputtxt))
        voice_button.place(x=50,y=180)
        Display = tk.Button(self, height = 2,width = 20,text ="Convert",command = lambda:Take_input(inputtxt))
        l.place(x=50, y=160)
        l1.place(x=115, y=230)
        inputtxt.place(x=50, y=250)
        Display.pack()

app = Tk_Manage()
app.geometry("800x750")
app.mainloop()


hi windy my
h
i
windy.webp
my.webp


In [1]:
pip install keras==2.13.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement keras==2.13.0 (from versions: 0.2.0, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.0.8, 1.1.0, 1.1.1, 1.1.2, 1.2.0, 1.2.1, 1.2.2, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.0.5, 2.0.6, 2.0.7, 2.0.8, 2.0.9, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.1.5, 2.1.6, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.2.4, 2.2.5, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.5.0rc0, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0rc3, 2.6.0, 2.7.0rc0, 2.7.0rc2, 2.7.0, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.10.0rc0, 2.10.0rc1, 2.10.0, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0rc3, 2.11.0, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.13.1rc0, 2.13.1rc1, 2.13.1, 2.14.0rc0, 2.14.0, 2.15.0rc0, 2.15.0rc1, 2.15.0, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.0.4, 3.0.5, 3.1.0, 3.1.1, 3.2.0, 3.2.1)
ERROR: No matching distribution found for keras==2.13.0


In [1]:
pip install tensorflow==2.13.0

  Using cached tensorflow-2.13.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp39-cp39-win_amd64.whl (276.5 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached numpy-1.24.3-cp39-cp39-win_amd64.whl (14.9 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.24.3 which is incompatible.
mediapipe 0.10.11 requires protobuf<4,>=3.11, but you have protobuf 4.25.3 which is incompatible.



  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-intel 2.16.1
    Uninstalling tensorflow-intel-2.16.1:
      Successfully uninstalled tensorflow-intel-2.16.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
